<a href="https://colab.research.google.com/github/trilgar/CNN_LSTM-heartbeat-classifier/blob/main/CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn,optim
import sys
from tqdm import tqdm
from torch.autograd import Variable

In [ ]:
from sklearn.utils import resample

def upsample(train_data):
   

    df_1=train_data[train_data[187]==1]
    df_2=train_data[train_data[187]==2]
    df_3=train_data[train_data[187]==3]
    df_4=train_data[train_data[187]==4]
    df_0=(train_data[train_data[187]==0]).sample(n=20000,random_state=42)

    df_1_upsample=resample(df_1,replace=True,n_samples=20000,random_state=123)
    df_2_upsample=resample(df_2,replace=True,n_samples=20000,random_state=124)
    df_3_upsample=resample(df_3,replace=True,n_samples=20000,random_state=125)
    df_4_upsample=resample(df_4,replace=True,n_samples=20000,random_state=126)

    train_df=pd.concat([df_0,df_1_upsample,df_2_upsample,df_3_upsample,df_4_upsample])
    
    return train_df

In [ ]:
def extend_with_generated(dataset):
    file_names = ['../input/heartbeat-dataset-gan-generated/heartbeat1.csv',
                 '../input/heartbeat-dataset-gan-generated/heartbeat2.csv',
                 '../input/heartbeat-dataset-gan-generated/heartbeat2_1.csv',
                 '../input/heartbeat-dataset-gan-generated/heartbeat2_2.csv',
                 '../input/heartbeat-dataset-gan-generated/heartbeat2_3.csv',
                 '../input/heartbeat-dataset-gan-generated/heartbeat3.csv',
                 '../input/heartbeat-dataset-gan-generated/heartbeat3_1.csv',
                 '../input/heartbeat-dataset-gan-generated/heartbeat3_2.csv',
                 '../input/heartbeat-dataset-gan-generated/heartbeat3_3.csv',
                 '../input/heartbeat-dataset-gan-generated/heartbeat4.csv']
    for path in file_names:
        generated = pd.read_csv(path, index_col= 0, header=0)
        generated.columns = dataset.columns
        dataset = pd.concat([dataset, generated])
    dataset = dataset.reset_index(drop=True)
    return dataset



In [ ]:
data = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header=None)
data

In [ ]:

data = extend_with_generated(data)
data

In [ ]:
data[187].value_counts()


In [ ]:
#define a Dataloader function
def my_DataLoader(train_root,test_root,batch_size = 100, val_split_factor = 0.2):

    train_df = pd.read_csv(train_root, header=None)
    test_df = pd.read_csv(test_root, header=None)
    
    

    train_data = train_df.to_numpy()
    test_data = test_df.to_numpy()

    train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(train_data[:, :-1]).float(),
                                                   torch.from_numpy(train_data[:, -1]).long(),)
    test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(test_data[:, :-1]).float(),
                                                  torch.from_numpy(test_data[:, -1]).long())

    train_len = train_data.shape[0]
    val_len = int(train_len * val_split_factor)
    train_len -= val_len

    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_loader, val_loader, test_loader

In [ ]:
#define the NET model

class  conv_net(nn.Module):

    def __init__(self, num_of_class):
        super(conv_net, self).__init__()
        self.lstm = nn.LSTM(input_size=187, hidden_size=64,
                          num_layers=5, batch_first=True) #lstm
        self.relu = nn.LeakyReLU(inplace=True)
        

        self.model = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

            nn.Conv1d(16, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

        )

        self.linear = nn.Sequential(
            nn.Linear(2944,500),
            nn.LeakyReLU(inplace=True),
            nn.Linear(500, num_of_class),

        )


    def forward(self,x):
        x = x.unsqueeze(1)
        h_0 = Variable(torch.zeros(5, x.size(0), 64)) 
        c_0 = Variable(torch.zeros(5, x.size(0), 64)) 
        
        output, (hn, cn) = self.lstm(x)
        hn = hn.view(-1, 64) 
        
        out = self.relu(x)
        
        x = self.model(out)
        x = x.view(x.size(0), -1)
        x = self.linear(x)

        return x


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, label_ranking_average_precision_score, label_ranking_loss, coverage_error 

def print_metric_evaluation(model, loader):
    model.eval()

    correct = 0
    total = len(loader.dataset)
    pred = []
    real = []
    
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pr = logits.argmax(dim=1)
            pred.extend(pr.detach().numpy())
            real.extend(y.detach().numpy())
            
#     print(pred)
#     print(real)
    
    print(classification_report(pred, real))

    

In [ ]:
batch_size=1000
lr = 3e-3
epochs = 50
torch.manual_seed(1234)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

accuracy = []
loss_ = []
    

def evalute(model, loader):
    model.eval()

    correct = 0
    total = len(loader.dataset)
    val_bar = tqdm(loader, file=sys.stdout)
    for x, y in val_bar:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
        correct += torch.eq(pred, y).sum().float().item()


    return correct / total

def main():

    train_loader, val_loader, test_loader = my_DataLoader('/kaggle/input/heartbeat/mitbih_train.csv',
                                                          '/kaggle/input/heartbeat/mitbih_test.csv',
                                                          batch_size=batch_size,
                                                          val_split_factor=0.2)

    model = conv_net(5).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criteon = nn.CrossEntropyLoss()

    best_acc, best_epoch = 0, 0
    global_step = 0
    


    for epoch in range(epochs):

        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, (x, y) in enumerate(train_bar):
            x, y = x.to(device), y.to(device)

            model.train()
            logits = model(x)
            loss = criteon(logits, y)
            
            loss_.append(loss.detach().numpy())

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

            global_step += 1

        if epoch % 1 == 0:

            val_acc = evalute(model, val_loader)
            accuracy.append(val_acc)
            print('val_acc = ',val_acc)
            if val_acc > best_acc:
                best_epoch = epoch
                best_acc = val_acc

                torch.save(model.state_dict(), 'best.mdl')

    print('best acc:', best_acc, 'best epoch:', best_epoch)

    model.load_state_dict(torch.load('best.mdl'))
    print('loaded from ckpt!')

    test_acc = evalute(model, test_loader)
    print('test acc:', test_acc)

    
    print_metric_evaluation(model, test_loader)


if __name__ == '__main__':
    main()


In [ ]:
import matplotlib.pyplot as plt

plt.title('accuracy')
plt.plot(accuracy, label="train")
plt.ylabel='accuracy'
plt.legend()
plt.show()

In [ ]:
plt.title('loss')
plt.plot(loss_, label='train')
plt.ylabel='accuracy'
plt.legend()
plt.show()